In [1]:
import re
from typing import Dict, Generator, List, Set

In [ ]:
from pyswip import Prolog

: 

In [14]:
def sample_generator() -> Generator[str, None, None]:
    data = ['a', 'b']
    for i in data:
        yield i

gen = sample_generator()

print(next(gen))
print(next(gen))
print(next(gen))

a
b


StopIteration: 

In [2]:
prolog = Prolog()
prolog.assertz("father(michael,john)")
prolog.assertz("father(michael,gina)")
list(prolog.query("father(michael,X)"))  # [{'X': 'john'}, {'X': 'gina'}]

[{'X': 'john'}, {'X': 'gina'}]

In [3]:
prolog1 = Prolog()
prolog2 = Prolog()

prolog1.consult("knowledge_base1.pl")
prolog2.consult("knowledge_base2.pl")

print(list(prolog1.query("father(michael,X)")))
print(list(prolog2.query("father(peter,X)")))
print(list(prolog1.query("father(michael,X)")))
print(list(prolog2.query("father(peter,X)")))

[]
[{'X': 'john'}, {'X': 'gina'}]
[]
[{'X': 'john'}, {'X': 'gina'}]


In [2]:
import pyswip, ctypes

class PrologMT(pyswip.Prolog):
    """Multi-threaded (one-to-one) pyswip.Prolog ad-hoc reimpl"""
    _swipl = pyswip.core._lib

    PL_thread_self = _swipl.PL_thread_self
    PL_thread_self.restype = ctypes.c_int

    PL_thread_attach_engine = _swipl.PL_thread_attach_engine
    PL_thread_attach_engine.argtypes = [ctypes.c_void_p]
    PL_thread_attach_engine.restype = ctypes.c_int

    @classmethod
    def _init_prolog_thread(cls):
        pengine_id = cls.PL_thread_self()
        if (pengine_id == -1):
            pengine_id = cls.PL_thread_attach_engine(None)
            print("{INFO} attach pengine to thread: %d" % pengine_id)
        if (pengine_id == -1):
            raise pyswip.prolog.PrologError("Unable to attach new Prolog engine to the thread")
        elif (pengine_id == -2):
            print("{WARN} Single-threaded swipl build, beware!")

    class _QueryWrapper(pyswip.Prolog._QueryWrapper):
        def __call__(self, *args, **kwargs):
            PrologMT._init_prolog_thread()
            return super().__call__(*args, **kwargs)

In [13]:
# For each Prolog() instance, a new thread is created
from threading import Thread

def run_prolog_thread():
    prolog = PrologMT()
    prolog.consult("knowledge_base1.pl")
    print(list(prolog.query("father(michael,X)")))

def run_prolog_thread2():
    prolog = PrologMT()
    prolog.consult("knowledge_base2.pl")
    print(str(list(prolog.query("father(peter,X)"))) + 'thread2')


thread = Thread(target=run_prolog_thread)
thread2 = Thread(target=run_prolog_thread2)
thread.start()
thread2.start()
thread.join()
thread2.join()

{INFO} attach pengine to thread: 20
{INFO} attach pengine to thread: 21
[{'X': 'bob'}, {'X': 'brian'}]thread2
[{'X': 'alison'}]


In [1]:
# Python using the swiplserver library
from swiplserver import PrologMQI, PrologThread

with PrologMQI() as mqi:
    with mqi.create_thread() as prolog_thread:
        result = prolog_thread.query("member(X, [first, second, third]).")
        print(result)

[{'X': 'first'}, {'X': 'second'}, {'X': 'third'}]


In [6]:
with PrologMQI() as mqi:
    with mqi.create_thread() as prolog_thread:
        prolog_thread.query("assert(father(michael,aaron)).")
        prolog_thread.query("assert(father(michael,alison)).")
        result = prolog_thread.query("father(michael,X).")
        print(result)

with PrologMQI() as mqi:
    with mqi.create_thread() as prolog_thread:
        prolog_thread.query("assert(father(peter,bob)).")
        prolog_thread.query("assert(father(peter,brian)).")
        result = prolog_thread.query("father(peter,X).")
        print(result)

[{'X': 'aaron'}, {'X': 'alison'}]
[{'X': 'bob'}, {'X': 'brian'}]


In [10]:
with PrologMQI() as mqi:
    with mqi.create_thread() as prolog_thread:
        prolog_thread.query("consult('knowledge_base1.pl').")
        result = prolog_thread.query("father(michael,X).")
        print(result)

with PrologMQI() as mqi:
    with mqi.create_thread() as prolog_thread:
        prolog_thread.query("consult('knowledge_base2.pl').")
        result = prolog_thread.query("father(peter,X).")
        print(result)

[{'X': 'aaron'}, {'X': 'alison'}]
[{'X': 'bob'}, {'X': 'brian'}]


In [13]:
def foo(x):
    print(x)


In [1]:
from multiprocessing import Pool

def f(x):
    return x*x

if __name__ == '__main__':
    with Pool(5) as p:
        print(p.map(f, [1, 2, 3]))